In [1]:
#Importe
import yfinance as yf
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np
import csv
from scipy.optimize import minimize
import math
import random
import scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import linkage, dendrogram
import riskfolio as rp
from sklearn.covariance import LedoitWolf
from scipy.spatial.distance import squareform
from riskfolio import HCPortfolio as hc
from pandas.tseries.offsets import MonthBegin
from pandas.tseries.offsets import MonthEnd

In [2]:
def generischer_optimierer (zielfunktion, cov_matrix):

    n = cov_matrix.shape[0]
    w0 = 1 / n

    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    bounds = [(0, 1)] * n

    res = minimize(zielfunktion, w0, method='SLSQP', bounds=bounds, constraints=constraints, options={'ftol': 1e-12, 'maxiter': 1000})

    return res.x

In [3]:
#Rückgabe gleicher Gewichte anhand der Anzahl der Unternehmen (Input Returns)

def equally_weighted(returns):
    anzahl_assets = returns.shape[1]

    w0 = 1/anzahl_assets
    gewichte = np.full(anzahl_assets, w0)

    return gewichte

In [4]:
#Minimum-Variance Portfolio
#Code adaptiert von https://medium.com/@BorisGerat/mean-variance-and-minimum-variance-portfolio-models-in-python-64a5c6b57b2d

def min_variance_opt(matrix):
    
    #Compute the minimum-variance portfolio weights subject to the constraints of:
    #- No short-selling (weights must be >= 0)
    #- Full investment (sum of weights must equal 1)
    
    number_of_assets = matrix.shape[0]

    # Define the objective function to minimize portfolio variance
    def objective(w):
        portfolio_variance = np.dot(w, matrix @ w)
        return portfolio_variance

    # Constraint: The sum of portfolio weights must be equal to 1
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]

    # Bounds: Each weight must be between 0 and 1 (no short-selling)
    bounds = [[0, 1] for _ in range(number_of_assets)]

    # Initial guess: Equal weights for all assets
    w0 = np.ones(number_of_assets) / number_of_assets

    # Solve the optimization problem using Sequential Least Squares Quadratic Programming (SLSQP)
    res = minimize(objective, w0, method='SLSQP', bounds=bounds, constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    
    return res.x  # Return the optimal weights

In [5]:
#Code von https://thequantmba.wordpress.com/2017/06/06/max-diversification-in-python/ adaptiert

def calc_max_diversification_ratio(weights, matrix):
    
    # gewichtete Volatilität
    average_vol = np.dot(np.sqrt(np.diag(matrix)), weights.T)
    
    # portfolio volatilität
    portfolio_variance = np.dot(weights, matrix @ weights)
    portfolio_vol = np.sqrt(portfolio_variance)
    
    #Bestimmung Ratio
    diversification_ratio = average_vol/portfolio_vol
    
    # Negativer Wert für Minimierung (Maximizieren = Minimieren -)
    return -diversification_ratio

In [6]:
#Code von https://thequantmba.wordpress.com/2017/06/06/max-diversification-in-python/ adaptiert

def max_diversification_portfolio(matrix):
    
    anzahl_assets = matrix.shape[0]
    
    w0 = 1/anzahl_assets
    w = np.full(anzahl_assets, w0)
    
    # Constraint: The sum of portfolio weights must be equal to 1
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    
    # Bounds: Each weight must be between 0 and 1 (no short-selling)
    bounds = [[0, 1] for _ in range(anzahl_assets)]
    
    res = minimize(calc_max_diversification_ratio, w, bounds=bounds, args=matrix, method='SLSQP', constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    return res.x

In [7]:
#Code adaptiert von https://www.kaggle.com/code/vijipai/lesson-4-traditional-portfolio-construction-method

def inverse_volatility_portfolio (matrix):
    
    standardabweichungen = np.sqrt(np.diagonal(matrix))
    invertierte_std = 1/standardabweichungen
    summe_invertierte_std = np.sum(invertierte_std)
    
    inverse_volatility_gewichte = invertierte_std / summe_invertierte_std
    
    return inverse_volatility_gewichte

In [8]:
#Code adaptiert von https://medium.com/@deepml1818/building-a-python-based-risk-parity-portfolio-for-trading-40441ecdd84d

def calculate_portfolio_volatility(weights, matrix):
    
    portfolio_variance = np.dot(weights, matrix @ weights)
    portfolio_volatility = np.sqrt(portfolio_variance)
    return portfolio_volatility

def calculate_risk_contribution(weights, matrix):
    portfolio_volatility = calculate_portfolio_volatility(weights, matrix)
    marginal_contribution_zaehler = np.dot(matrix, weights)
    risk_contribution = np.multiply(weights, marginal_contribution_zaehler) / portfolio_volatility
    return risk_contribution

def risk_parity_objective(weights, matrix):
    risk_contribution = calculate_risk_contribution(weights, matrix)
    target_risk = np.mean(risk_contribution)
    return np.sum((risk_contribution - target_risk) ** 2)

In [9]:
def get_risk_parity_weights(matrix):
    
    number_of_assets = matrix.shape[0]
    w0 = 1/number_of_assets
    initial_weights = np.full(number_of_assets, w0)
    
    constraints = ({'type': 'eq', 'fun': lambda initial_weights: np.sum(initial_weights) - 1})
    bounds = [[0, 1] for _ in range(number_of_assets)]
    
    result = minimize(risk_parity_objective, initial_weights, args=(matrix), method='SLSQP', bounds=bounds, constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    
    return result.x

In [10]:
#Aus dem Paper von Lopez De Prado (2016)
def getIVP(cov,**kargs): # Compute the inverse-variance portfolio 
    ivp=1./np.diag(cov) 
    ivp/=ivp.sum() 
    return ivp

In [11]:
#------------------------------------------------------------------------------ 
def getClusterVar(cov,cItems): # Compute variance per cluster 
    cov_=cov.loc[cItems,cItems] # matrix slice 
    w_=getIVP(cov_).reshape(-1,1) 
    cVar=np.dot(np.dot(w_.T,cov_),w_)[0,0] 
    return cVar 

In [12]:
#------------------------------------------------------------------------------ 
def getQuasiDiag(link): # Sort clustered items by distance 
    link=link.astype(int) 
    sortIx=pd.Series([link[-1,0],link[-1,1]]) 
    numItems=link[-1,3] # number of original items 
    while sortIx.max()>=numItems: 
        sortIx.index=range(0,sortIx.shape[0]*2,2) # make space 
        df0=sortIx[sortIx>=numItems] # find clusters
        i=df0.index;j=df0.values-numItems 
        sortIx[i]=link[j,0] # item 1 
        df0=pd.Series(link[j,1],index=i+1) 
        sortIx = pd.concat([sortIx, df0]) # item 2  ####Anpassung wegen neuer Pandas-Version!
        sortIx=sortIx.sort_index() # re-sort 
        sortIx.index=range(sortIx.shape[0]) # re-index 
    return sortIx.tolist() 

In [13]:
#------------------------------------------------------------------------------ 
def getRecBipart(cov,sortIx): # Compute HRP allocation
    w = pd.Series(1,index=sortIx) 
    cItems=[sortIx] # initialize all items in one cluster 
    while len(cItems)>0: 
        #Bi-Section der Cluster
        cItems = [i[j:k] 
                  for i in cItems 
                  for j, k in ((0, len(i)//2), (len(i)//2, len(i))) 
                  if len(i) > 1]
        for i in range(0, len(cItems), 2): # parse in pairs 
            cItems0=cItems[i] # cluster 1 
            cItems1=cItems[i+1] # cluster 2 
            cVar0=getClusterVar(cov,cItems0) 
            cVar1=getClusterVar(cov,cItems1) 
            alpha=1-cVar0/(cVar0+cVar1) 
            w[cItems0]*=alpha # weight 1 
            w[cItems1]*=1-alpha # weight 2 
    return w 

In [14]:
#------------------------------------------------------------------------------ 
def correlDist(corr): 
    # A distance matrix based on correlation, where 0<=d[i,j]<=1 
    # This is a proper distance metric 
    dist=((1-corr)/2.)**.5 # distance matrix 
    
    #Adjustierung zur condensed_distance_matrix
    condensed_distance_matrix = squareform(X=dist, checks=False)
    
    return condensed_distance_matrix

In [15]:
def hrp_main(corr, cov, returns): 
    
    #3) cluster 
    dist=correlDist(corr) 
    link=sch.linkage(dist,'single') 
    sortIx=getQuasiDiag(link) 
    sortIx=corr.index[sortIx].tolist() # recover labels 
    df0=corr.loc[sortIx,sortIx] # reorder 
    
    #4) Capital allocation 
    hrp=getRecBipart(cov,sortIx) 
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    hrp_sortiert = hrp.reindex(original_reihenfolge)
    
    array = hrp_sortiert.T.values

    return array

In [16]:
#Es wird auf die Implementierung der Riskfolio-Lib zurückgegriffen
#Code adaptiert von #Code adaptiert von https://medium.com/@orenji.eirl/hierarchical-equal-risk-contribution-with-python-and-riskfolio-lib-ec45dd0f9899

def herc (returns):

    # Building the portfolio object
    port = hc(returns=returns)

    # Estimate optimal portfolio:

    model='HERC'
    correlation = 'pearson'
    rm = 'MV'
    rf = 0 
    linkage = 'ward' 
    max_k = 10 
    leaf_order = True 

    gewichte = port.optimization(model=model,
                          correlation=correlation,
                          rm=rm,
                          rf=rf,
                          linkage=linkage,
                          max_k=max_k,
                          leaf_order=leaf_order)
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    gewichte_sortiert = gewichte.reindex(original_reihenfolge)
    
    array = gewichte_sortiert.values.flatten()

    return array

In [17]:
def herc_ledoit (returns):

    #Building the portfolio object
    port = hc(returns=returns)
    
    shrinkage_herc = LedoitWolf().fit(returns.values)
    
    shrinkage_matrix_herc = pd.DataFrame(
        shrinkage_herc.covariance_,
        index = returns.columns,
        columns = returns.columns)

    #Estimate optimal portfolio:

    model='HERC'
    #correlation = 'pearson'
    rm = 'MV'
    rf = 0 
    linkage = 'ward' 
    max_k = 10 
    leaf_order = True 

    gewichte = port.optimization(model=model,
                          custom_cov = shrinkage_matrix_herc,
                          codependence='custom_cov',
                          rm=rm,
                          rf=rf,
                          method_cov='custom_cov',   #gemäß https://riskfolio-lib.readthedocs.io/en/latest/hcportfolio.html
                          linkage=linkage,
                          max_k=max_k,
                          leaf_order=leaf_order)
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    gewichte_sortiert = gewichte.reindex(original_reihenfolge)
    
    array = gewichte_sortiert.values.flatten()

    return array

# Checks S&P500

In [18]:
#Bearbeitete Wharton-Daten (Gesamtbestand)

df_Wharton = pd.read_csv(
    'WhartonFinal_Neu.csv',
    sep=',',
    encoding='utf-8',
    index_col=0,
    decimal = '.',
    low_memory=False,
    parse_dates=['Date'],
    dayfirst=True
)

In [20]:
#Tägliche Renditen für den gesamten Zeitraum von 20 Jahren SP500

returns = df_Wharton.pct_change()
returns = returns.where(~df_Wharton.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                 8303     15604     15618     16179     16183     16291  \
Date                                                                     
2005-06-02 -0.005128 -0.001771  0.006711  0.000690 -0.010465 -0.004525   
2005-06-03 -0.023110 -0.019516  0.008533 -0.000689 -0.001175 -0.006818   
2005-06-06 -0.023657 -0.000658  0.037678 -0.044828  0.014118  0.002288   
2005-06-07 -0.024230  0.002469 -0.007517  0.002888  0.005800  0.011416   
2005-06-08 -0.027970 -0.003284 -0.005135  0.006479  0.014994  0.006772   
...              ...       ...       ...       ...       ...       ...   
2025-05-26       NaN       NaN  0.013928       NaN -0.001542       NaN   
2025-05-27       NaN       NaN -0.003930       NaN  0.010039       NaN   
2025-05-28       NaN       NaN  0.004102       NaN  0.001529       NaN   
2025-05-29       NaN       NaN  0.010933       NaN  0.016794       NaN   
2025-05-30       NaN       NaN -0.016665       NaN  0.012763       NaN   

     

In [21]:
returns.to_csv("Wharton_Debug8.csv")

In [22]:
#Ein einzelner Zeitraum (1 Trainingsfenster + anschließendes Testzeitfenster)
#Die Richtigkeit der Methoden soll getestet werden

#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2016-09-01')
end_datum = pd.Timestamp('2019-09-30')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 36
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)
    
print(training_anfang)
    
#Definition des Trainingsdatensatzes und des Testdatensatzes
training_daten = returns.loc[training_anfang:training_ende]
test_daten = returns.loc[test_anfang:test_ende]
training_daten = training_daten.dropna(axis=1)
    
#Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
max_null_tage = 189
null_zaehler = (training_daten == 0).sum()
valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
training_daten = training_daten[valide_tickers]
    
#Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
test_daten = test_daten[vorhandene_assets]
test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
#Erzeugung Matrizen
cov_matrix = training_daten.cov()
corr_matrix = training_daten.corr()


2016-09-01 00:00:00


In [23]:
training_daten.info

<bound method DataFrame.info of                15618     16183     16305     16356     16610     17287  \
Date                                                                     
2016-09-01 -0.012929 -0.020842 -0.039624 -0.024580  0.009132  0.014729   
2016-09-02  0.046946  0.014056  0.039510 -0.019139  0.050712  0.002765   
2016-09-05  0.006560  0.000000 -0.001009  0.004878 -0.043373 -0.006548   
2016-09-06  0.019458  0.034851  0.028822  0.049110  0.025588 -0.003296   
2016-09-07 -0.028813 -0.003444 -0.028014 -0.008252 -0.001094 -0.025409   
...              ...       ...       ...       ...       ...       ...   
2019-08-26 -0.005971 -0.004772  0.000000 -0.000750  0.031761  0.010360   
2019-08-27 -0.021091 -0.001308 -0.021391 -0.014254  0.001160 -0.027832   
2019-08-28  0.013702  0.003492  0.016038 -0.010654  0.057557  0.054746   
2019-08-29 -0.004688 -0.004785 -0.017584 -0.000462  0.016125  0.028095   
2019-08-30  0.033426  0.005245  0.041966  0.012929  0.024266 -0.033349   

     

In [24]:
#Summe der letzten Zeile
training_daten.iloc[-1].sum()

1.7437441959262006

In [23]:
##### ÜBERPRÜFUNG ERSTELLUNG TRAINGSZEITFENSTER!

#Start- und Enddatum sind 01.09.2016 und 30.08.2019
#Erreicht: Ja

#Zielwert Zeilen laut manueller Berechnung in Excel: 782
#Erreicht: Ja 

#Zielwert Spalten laut manueller Berechnung in Excel: 107
#Erreicht: Ja

#Summe der letzten Zeile als Überprüfungswert in Excel: 1.743744196
#Erreicht: Ja

#Berechnung Trainings-Zeitfenster in Ordnung

In [25]:
test_daten.info

<bound method DataFrame.info of                15618     16183     16305     16356     16610     17287  \
Date                                                                     
2019-09-02 -0.007301 -0.003043 -0.013599 -0.013220 -0.033704  0.003833   
2019-09-03  0.001961  0.012211  0.013325  0.014937 -0.007906 -0.023866   
2019-09-04  0.003559  0.012495 -0.000976  0.020482  0.008406 -0.049389   
2019-09-05  0.024115 -0.004255  0.026259 -0.108534 -0.010103 -0.015947   
2019-09-06  0.006406  0.014957  0.016825  0.005170 -0.013742 -0.029273   
2019-09-09 -0.006408 -0.010526  0.003185  0.007632 -0.007639 -0.048465   
2019-09-10 -0.000476  0.000000  0.002116 -0.003787 -0.036606 -0.026599   
2019-09-11 -0.003941 -0.026383 -0.014348  0.011405  0.010348 -0.003488   
2019-09-12  0.015783  0.011801  0.021236  0.030070  0.057219  0.052509   
2019-09-13  0.021016  0.010367  0.030483  0.006188 -0.002586 -0.079268   
2019-09-16  0.053241  0.029072  0.030299  0.006465  0.018408 -0.006020   
2019-0

In [26]:
#Summe der letzten Zeile
test_daten.iloc[-1].sum()

-0.5602333099658309

In [27]:
##### ÜBERPRÜFUNG ERSTELLUNG TESTZEITFENSTER!

#Start- und Enddatum sind 02.09.2019 und 30.09.2019
#Erreicht: Ja

#Zielwert Zeilen laut manueller Berechnung in Excel: 21
#Erreicht: Ja 

#Zielwert Spalten laut manueller Berechnung in Excel: 107
#Erreicht: Ja

#Summe der letzten Zeile als Überprüfungswert in Excel: -0,56023331
#Erreicht: Ja

#Berechnung Trainings-Zeitfenster in Ordnung

In [28]:
def ledoit_shrink (returns):
    
    shrinkage = LedoitWolf().fit(returns.values)
    
    shrinkage_matrix = pd.DataFrame(
        shrinkage.covariance_,
        index = returns.columns,
        columns = returns.columns)
    
    return shrinkage_matrix

In [29]:
cov_matrix

,15618,16183,16305,16356,16610,17287,17315,17947,24946,60918,...,318215,318385,318455,319477,352362,352364,320485,320490,320733,321670
15618,0.000281,0.000083,0.000231,0.000092,0.000022,-7.325492e-05,0.000054,-2.581396e-05,0.000133,0.000002,...,0.000052,0.000062,1.013457e-05,3.326703e-07,0.000077,0.000081,2.949404e-05,0.000040,0.000028,5.163262e-05
16183,0.000083,0.000139,0.000083,0.000084,0.000014,-3.299068e-05,0.000020,-2.353021e-06,0.000061,0.000004,...,0.000024,0.000012,1.984025e-05,3.744318e-05,0.000039,0.000034,1.366909e-06,0.000027,0.000006,4.197693e-05
16305,0.000231,0.000083,0.000345,0.000101,0.000038,-3.998286e-05,0.000070,-2.017408e-05,0.000155,-0.000005,...,0.000063,0.000055,1.256788e-05,-3.970610e-05,0.000089,0.000080,4.255974e-05,0.000050,0.000018,4.995450e-05
16356,0.000092,0.000084,0.000101,0.000195,0.000010,-5.271963e-05,0.000018,7.294348e-07,0.000090,-0.000003,...,0.000016,0.000038,1.668333e-05,9.641947e-06,0.000055,0.000043,1.011261e-05,0.000031,0.000016,2.440103e-05
16610,0.000022,0.000014,0.000038,0.000010,0.000531,3.018105e-04,0.000206,2.394078e-04,0.000059,0.000060,...,0.000025,0.000032,6.024452e-06,-4.648045e-06,0.000072,0.000057,2.164913e-05,0.000042,-0.000015,5.473093e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352364,0.000081,0.000034,0.000080,0.000043,0.000057,1.225207e-05,0.000106,2.613812e-05,0.000067,0.000035,...,0.000020,0.000014,1.448166e-05,6.441678e-06,0.000063,0.000236,2.156818e-05,0.000019,0.000014,5.069981e-05
320485,0.000029,0.000001,0.000043,0.000010,0.000022,6.897245e-05,0.000020,-2.388252e-05,0.000025,-0.000004,...,-0.000001,0.000029,6.435136e-07,-2.404985e-05,0.000014,0.000022,6.329458e-04,0.000018,0.000001,5.789401e-07
320490,0.000040,0.000027,0.000050,0.000031,0.000042,1.668275e-05,0.000048,1.598272e-05,0.000035,-0.000006,...,0.000051,0.000071,2.698033e-06,-1.242766e-05,0.000031,0.000019,1.796825e-05,0.000820,0.000001,2.067749e-05
320733,0.000028,0.000006,0.000018,0.000016,-0.000015,-1.707334e-05,0.000009,1.095446e-05,0.000017,-0.000020,...,0.000042,0.000024,8.629040e-06,3.100548e-05,0.000017,0.000014,1.440432e-06,0.000001,0.000146,1.290910e-05


In [30]:
cov_ledoit = ledoit_shrink (training_daten)
cov_ledoit

,15618,16183,16305,16356,16610,17287,17315,17947,24946,60918,...,318215,318385,318455,319477,352362,352364,320485,320490,320733,321670
15618,0.000295,0.000076,0.000212,0.000084,0.000020,-6.711798e-05,0.000049,-2.365140e-05,0.000122,0.000002,...,0.000048,0.000056,9.285548e-06,3.048008e-07,0.000070,0.000074,2.702318e-05,0.000036,0.000026,4.730710e-05
16183,0.000076,0.000166,0.000076,0.000077,0.000012,-3.022688e-05,0.000018,-2.155896e-06,0.000056,0.000003,...,0.000022,0.000011,1.817813e-05,3.430637e-05,0.000036,0.000031,1.252396e-06,0.000025,0.000005,3.846031e-05
16305,0.000212,0.000076,0.000354,0.000093,0.000035,-3.663329e-05,0.000064,-1.848399e-05,0.000142,-0.000004,...,0.000058,0.000050,1.151500e-05,-3.637972e-05,0.000081,0.000074,3.899429e-05,0.000046,0.000016,4.576955e-05
16356,0.000084,0.000077,0.000093,0.000217,0.000010,-4.830304e-05,0.000017,6.683263e-07,0.000083,-0.000002,...,0.000014,0.000035,1.528568e-05,8.834192e-06,0.000050,0.000039,9.265427e-06,0.000029,0.000015,2.235683e-05
16610,0.000020,0.000012,0.000035,0.000010,0.000525,2.765263e-04,0.000188,2.193514e-04,0.000054,0.000055,...,0.000023,0.000029,5.519753e-06,-4.258655e-06,0.000066,0.000052,1.983547e-05,0.000038,-0.000014,5.014584e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352364,0.000074,0.000031,0.000074,0.000039,0.000052,1.122565e-05,0.000097,2.394839e-05,0.000061,0.000032,...,0.000019,0.000013,1.326846e-05,5.902025e-06,0.000058,0.000255,1.976130e-05,0.000018,0.000012,4.645243e-05
320485,0.000027,0.000001,0.000039,0.000009,0.000020,6.319427e-05,0.000019,-2.188176e-05,0.000023,-0.000004,...,-0.000001,0.000027,5.896031e-07,-2.203507e-05,0.000013,0.000020,6.183305e-04,0.000016,0.000001,5.304393e-07
320490,0.000036,0.000025,0.000046,0.000029,0.000038,1.528515e-05,0.000044,1.464376e-05,0.000032,-0.000006,...,0.000046,0.000065,2.472005e-06,-1.138653e-05,0.000029,0.000018,1.646296e-05,0.000789,0.000001,1.894523e-05
320733,0.000026,0.000005,0.000016,0.000015,-0.000014,-1.564302e-05,0.000008,1.003675e-05,0.000016,-0.000018,...,0.000038,0.000022,7.906141e-06,2.840799e-05,0.000015,0.000012,1.319760e-06,0.000001,0.000172,1.182764e-05


In [31]:
corr_matrix

,15618,16183,16305,16356,16610,17287,17315,17947,24946,60918,...,318215,318385,318455,319477,352362,352364,320485,320490,320733,321670
15618,1.000000,0.419197,0.742512,0.391959,0.057463,-0.123539,0.127914,-0.058601,0.482846,0.003999,...,0.143889,0.130312,0.026788,0.000441,0.444800,0.314011,0.069986,0.082508,0.139505,0.188043
16183,0.419197,1.000000,0.380312,0.509213,0.050118,-0.079054,0.067744,-0.007590,0.312208,0.008687,...,0.093397,0.036405,0.074516,0.070467,0.322742,0.188698,0.004609,0.080844,0.039246,0.217224
16305,0.742512,0.380312,1.000000,0.390679,0.088982,-0.060811,0.149135,-0.041303,0.507029,-0.006778,...,0.156301,0.104328,0.029960,-0.047429,0.463360,0.281745,0.091078,0.093454,0.078535,0.164076
16356,0.391959,0.509213,0.390679,1.000000,0.032506,-0.106631,0.052126,0.001986,0.392832,-0.005077,...,0.051457,0.096346,0.052889,0.015316,0.381218,0.198143,0.028780,0.078060,0.093765,0.106582
16610,0.057463,0.050118,0.088982,0.032506,1.000000,0.369939,0.354039,0.395018,0.154018,0.070318,...,0.050539,0.049421,0.011574,-0.004475,0.303977,0.159493,0.037338,0.063604,-0.054833,0.144875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352364,0.314011,0.188698,0.281745,0.198143,0.159493,0.022517,0.273941,0.064664,0.263122,0.060672,...,0.061058,0.031843,0.041715,0.009298,0.399918,1.000000,0.055773,0.043929,0.072730,0.201221
320485,0.069986,0.004609,0.091078,0.028780,0.037338,0.077446,0.032166,-0.036098,0.060358,-0.004523,...,-0.002425,0.041492,0.001133,-0.021209,0.054192,0.055773,1.000000,0.024947,0.004735,0.001404
320490,0.082508,0.080844,0.093454,0.078060,0.063604,0.016462,0.066036,0.021229,0.074318,-0.006056,...,0.081591,0.088071,0.004173,-0.009631,0.105892,0.043929,0.024947,1.000000,0.003907,0.044062
320733,0.139505,0.039246,0.078535,0.093765,-0.054833,-0.039884,0.029965,0.034447,0.087694,-0.044667,...,0.160411,0.069605,0.031594,0.056884,0.135434,0.072730,0.004735,0.003907,1.000000,0.065123


In [32]:
#aus der geschrumpften Matrix die Korrelationsmatrix bauen
standardabweichung = np.sqrt(np.diag(cov_ledoit))
corr_ledoit = cov_ledoit / np.outer(standardabweichung, standardabweichung)
corr_ledoit

,15618,16183,16305,16356,16610,17287,17315,17947,24946,60918,...,318215,318385,318455,319477,352362,352364,320485,320490,320733,321670
15618,1.000000,0.342718,0.653980,0.331690,0.051601,-0.113351,0.115557,-0.053075,0.419228,0.003675,...,0.128582,0.118461,0.024019,0.000407,0.351472,0.269923,0.063219,0.075063,0.114715,0.163132
16183,0.342718,1.000000,0.314768,0.404932,0.042291,-0.068160,0.057510,-0.006460,0.254728,0.007502,...,0.078429,0.031099,0.062785,0.061137,0.239647,0.152424,0.003912,0.069114,0.030326,0.177084
16305,0.653980,0.314768,1.000000,0.334692,0.080891,-0.056485,0.136393,-0.037870,0.445665,-0.006305,...,0.141399,0.096011,0.027195,-0.044331,0.370662,0.245180,0.083289,0.086072,0.065378,0.144098
16356,0.331690,0.404932,0.334692,1.000000,0.028392,-0.095163,0.045804,0.001750,0.331752,-0.004538,...,0.044726,0.085191,0.046126,0.013755,0.292998,0.165668,0.025286,0.069075,0.074995,0.089935
16610,0.051601,0.042291,0.080891,0.028392,1.000000,0.350339,0.330120,0.369267,0.138023,0.066697,...,0.046614,0.046371,0.010711,-0.004264,0.247918,0.141507,0.034812,0.059725,-0.046539,0.129722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352364,0.269923,0.152424,0.245180,0.165668,0.141507,0.020413,0.244514,0.057865,0.225718,0.055087,...,0.053909,0.028600,0.036955,0.008482,0.312224,1.000000,0.049777,0.039487,0.059090,0.172474
320485,0.063219,0.003912,0.083289,0.025286,0.034812,0.073778,0.030171,-0.033945,0.054411,-0.004316,...,-0.002250,0.039162,0.001054,-0.020331,0.044460,0.049777,1.000000,0.023565,0.004042,0.001264
320490,0.075063,0.069114,0.086072,0.069075,0.059725,0.015794,0.062383,0.020106,0.067475,-0.005819,...,0.076242,0.083720,0.003912,-0.009298,0.087497,0.039487,0.023565,1.000000,0.003360,0.039972
320733,0.114715,0.030326,0.065378,0.074995,-0.046539,-0.034587,0.025586,0.029487,0.071964,-0.038796,...,0.135484,0.059805,0.026775,0.049639,0.101148,0.059090,0.004042,0.003360,1.000000,0.053397


In [33]:
is_in_range = np.all((corr_matrix >= -1) & (corr_matrix <= 1))

print(is_in_range)

True


In [34]:
tol = 1e-12
is_in_range_ledoit = np.all((corr_ledoit >= -1) & (corr_ledoit <= 1 + tol))

print(is_in_range_ledoit)

True


In [34]:
#ÜBERPRÜFUNG KOVARIANZ_MATRIZEN

#Alle 4 Kovarianz- und Korrelationsmatrizen haben die gleiche Größe (NxN)
#Erreicht: Ja

#Korrelationsmatrizen haben Werte zwischen -1 und 1 und haben 1 auf der Diagonale
#Erreicht: Ja

#Kovarianz und Korrelationsmatrizen unter Ledoit unterscheiden sich von den gewöhnlichen Matrizen
#Erreicht: Ja

In [35]:
gewichte_EW = equally_weighted(training_daten)
gewichte_MV = min_variance_opt(cov_matrix)
gewichte_MD = max_diversification_portfolio(cov_matrix)
gewichte_IV = inverse_volatility_portfolio(cov_matrix)
gewichte_ERC = get_risk_parity_weights(cov_matrix)
gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
gewichte_herc = herc (training_daten)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.8664048718996837e-05
            Iterations: 209
            Function evaluations: 22572
            Gradient evaluations: 209
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.5593316093572245
            Iterations: 80
            Function evaluations: 8740
            Gradient evaluations: 80
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.166062151984071e-09
            Iterations: 33
            Function evaluations: 3564
            Gradient evaluations: 33


In [36]:
print(1/107)
print(gewichte_EW.min())
print(gewichte_EW.sum())

0.009345794392523364
0.009345794392523364
1.0


In [37]:
#ÜBERPRÜFUNG METHODE EQUALLY WEIGHTED

#Minimales Gewicht muss 1/N entsprechen
#Erreicht: Ja

#Gewichte müssen aufsummiert 1 ergeben
#Erreicht: Ja

In [38]:
wert = gewichte_MV.min()
print(f"{wert:.12f}")
wert = gewichte_MV.sum()
print(f"{wert:.12f}")

0.000000000000
1.000000000000


In [39]:
#ÜBERPRÜFUNG METHODE MINIMUM-VARIANCE

#Optimierer terminiert erfolgreich mit einem Zielwert über 0
#Erreicht: Ja

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [39]:
wert = gewichte_MD.min()
print(f"{wert:.12f}")
wert = gewichte_MD.sum()
print(f"{wert:.12f}")

0.000000000000
1.000000000000


In [40]:
#ÜBERPRÜFUNG METHODE MAXIMUM-DIVERSIFICATION

#Optimierer terminiert erfolgreich mit einem Zielwert unter 0
#Erreicht: Ja

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [41]:
wert = gewichte_IV.min()
print(f"{wert:.12f}")
wert = gewichte_IV.sum()
print(f"{wert:.12f}")

0.003835780833
1.000000000000


In [42]:
#ÜBERPRÜFUNG METHODE INVERSE-VOLATILITY

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [43]:
wert = gewichte_ERC.min()
print(f"{wert:.12f}")
wert = gewichte_ERC.sum()
print(f"{wert:.12f}")

0.004209234648
1.000000000000


In [44]:
#ÜBERPRÜFUNG METHODE EQUAL-RISK-CONTRIBUTION

#Optimierer terminiert erfolgreich mit einem Zielwert über 0
#Erreicht: Ja

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [45]:
wert = gewichte_hrp.min()
print(f"{wert:.12f}")
wert = gewichte_hrp.sum()
print(f"{wert:.12f}")

0.001236909429
1.000000000000


In [46]:
#ÜBERPRÜFUNG METHODE HIERARCHICAL RISK PARITY

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [47]:
wert = gewichte_herc.min()
print(f"{wert:.12f}")
wert = gewichte_herc.sum()
print(f"{wert:.12f}")

0.000127902073
1.000000000000


In [48]:
#ÜBERPRÜFUNG METHODE HIERARCHICAL EQUAL RISK CONTRIBUTION

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [49]:
np.savetxt("Wharton_test_gewichte_EW.csv", gewichte_EW, delimiter=",", fmt="%.12f")
np.savetxt("Wharton_test_gewichte_MV.csv", gewichte_MV, delimiter=",", fmt="%.12f")
np.savetxt("Wharton_test_gewichte_MD.csv", gewichte_MD, delimiter=",", fmt="%.12f")
np.savetxt("Wharton_test_gewichte_IV.csv", gewichte_IV, delimiter=",", fmt="%.12f")
np.savetxt("Wharton_test_gewichte_ERC.csv", gewichte_ERC, delimiter=",", fmt="%.12f")
np.savetxt("Wharton_test_gewichte_HERC.csv", gewichte_herc, delimiter=",", fmt="%.12f")
np.savetxt("Wharton_test_gewichte_HRP.csv", gewichte_hrp, delimiter=",", fmt="%.12f")

In [50]:
#TEST DER ERGEBNISSE (RENDITEN AUF TEST-DATEN ANHAND GEWICHTE)

returns_EW = test_daten @ gewichte_EW
returns_MV = test_daten @ gewichte_MV
returns_MD = test_daten @ gewichte_MD
returns_IV = test_daten @ gewichte_IV
returns_ERC = test_daten @ gewichte_ERC
returns_HRP = test_daten @ gewichte_hrp
returns_HERC = test_daten @ gewichte_herc

print(returns_EW)
print(returns_MV)
print(returns_MD)
print(returns_IV)
print(returns_ERC)
print(returns_HRP)
print(returns_HERC)

Date
2019-09-02   -0.009196
2019-09-03    0.001457
2019-09-04   -0.000431
2019-09-05    0.008636
2019-09-06    0.001248
2019-09-09    0.002003
2019-09-10   -0.003995
2019-09-11    0.001803
2019-09-12    0.015048
2019-09-13    0.003032
2019-09-16    0.013283
2019-09-17   -0.015342
2019-09-18   -0.000759
2019-09-19   -0.002452
2019-09-20   -0.005131
2019-09-23    0.000447
2019-09-24    0.000000
2019-09-25   -0.008073
2019-09-26    0.005027
2019-09-27   -0.002784
2019-09-30   -0.005236
dtype: float64
Date
2019-09-02   -0.009993
2019-09-03   -0.000176
2019-09-04   -0.000022
2019-09-05    0.008154
2019-09-06    0.002380
2019-09-09   -0.000295
2019-09-10    0.001439
2019-09-11   -0.001703
2019-09-12    0.011647
2019-09-13    0.003444
2019-09-16    0.012438
2019-09-17   -0.006995
2019-09-18   -0.000694
2019-09-19   -0.006877
2019-09-20    0.012535
2019-09-23   -0.004394
2019-09-24    0.000000
2019-09-25   -0.001067
2019-09-26    0.001930
2019-09-27   -0.003998
2019-09-30    0.001700
dtype: fl

In [51]:
#ÜBERPRÜFUNG GEWICHTE UND RETURNS

#Alle Methoden geben 107 Werte (Gewichte) zurück (Array-Längen 107)
#Erreicht: Ja

#Manuell ermittelte Returns stimmen mit der Implementierung überein
#Erreicht: Ja

In [52]:
gewichte_MV = min_variance_opt(cov_ledoit)
gewichte_MD = max_diversification_portfolio(cov_ledoit)
gewichte_IV = inverse_volatility_portfolio(cov_ledoit)
gewichte_ERC = get_risk_parity_weights(cov_ledoit)
gewichte_hrp = hrp_main(corr_ledoit, cov_ledoit, training_daten)
gewichte_herc = herc_ledoit (training_daten)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.8492685577698885e-05
            Iterations: 188
            Function evaluations: 20304
            Gradient evaluations: 188
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.663812261341039
            Iterations: 81
            Function evaluations: 8861
            Gradient evaluations: 81
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.3290392460575575e-09
            Iterations: 33
            Function evaluations: 3564
            Gradient evaluations: 33


In [53]:
wert = gewichte_MV.min()
print(f"{wert:.12f}")
wert = gewichte_MV.sum()
print(f"{wert:.12f}")

0.000000000000
1.000000000000


In [54]:
#ÜBERPRÜFUNG METHODE MINIMUM-VARIANCE UNTER LEDOIT

#Optimierer terminiert erfolgreich mit einem Zielwert über 0
#Erreicht: Ja

#Zielwert des Optimierers unterscheidet sich unter Ledoit von normaler Berechnung
#Erreicht: Ja

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [55]:
wert = gewichte_MD.min()
print(f"{wert:.12f}")
wert = gewichte_MD.sum()
print(f"{wert:.12f}")

0.000000000000
1.000000000000


In [56]:
#ÜBERPRÜFUNG METHODE MAXIMUM-DIVERSIFICATION UNTER LEDOIT

#Optimierer terminiert erfolgreich mit einem Zielwert unter 0
#Erreicht: Ja

#Zielwert des Optimierers unterscheidet sich unter Ledoit von normaler Berechnung
#Erreicht: Ja

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [57]:
wert = gewichte_IV.min()
print(f"{wert:.12f}")
wert = gewichte_IV.sum()
print(f"{wert:.12f}")

0.004079067780
1.000000000000


In [58]:
#ÜBERPRÜFUNG METHODE INVERSE-VOLATILITY UNTER LEDOIT

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [59]:
wert = gewichte_ERC.min()
print(f"{wert:.12f}")
wert = gewichte_ERC.sum()
print(f"{wert:.12f}")

0.004277580846
1.000000000000


In [60]:
#ÜBERPRÜFUNG METHODE EQUAL-RISK-CONTRIBUTION UNTER LEDOIT

#Optimierer terminiert erfolgreich mit einem Zielwert über 0
#Erreicht: Ja

#Zielwert des Optimierers unterscheidet sich unter Ledoit von normaler Berechnung
#Erreicht: Ja

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [61]:
wert = gewichte_hrp.min()
print(f"{wert:.12f}")
wert = gewichte_hrp.sum()
print(f"{wert:.12f}")

0.001262872087
1.000000000000


In [62]:
#ÜBERPRÜFUNG METHODE HRP UNTER LEDOIT

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [63]:
wert = gewichte_herc.min()
print(f"{wert:.12f}")
wert = gewichte_herc.sum()
print(f"{wert:.12f}")

0.000294057276
1.000000000000


In [64]:
#ÜBERPRÜFUNG METHODE HERC UNTER LEDOIT

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja